In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("./data/features3_train.csv")
X = data[data.columns[1:-1]]
y = data['VV']

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33)

In [4]:
from Python.sklearn.data_split import *
X_train_bal, y_train_bal = balance_sample(X_train, y_train)
X_train_bal.shape


(34236, 26)

In [14]:
import xgboost
xg_reg = xgboost.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3,
                              learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)
dtrain = xgboost.DMatrix(np.asmatrix(X_train_bal), label=y_train_bal)
dtest = xgboost.DMatrix(np.asmatrix(X_valid), label=y_valid)
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgboost.cv(dtrain=dtrain, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="auc", as_pandas=True, seed=123)
cv_results.head()

[14:30:10] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:30:10] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:30:10] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.666624,0.007181,0.663417,0.003419
1,0.752452,0.023378,0.749826,0.028219
2,0.772304,0.020354,0.769272,0.024942
3,0.790559,0.028694,0.785803,0.032768
4,0.824705,0.013148,0.819995,0.014792


In [15]:
from sklearn.metrics import f1_score
xg_reg = xgboost.train(params=params, dtrain=dtrain, num_boost_round=10)
y_valid_pred = xg_reg.predict(dtest)
y_valid_pred = [0 if x < 0.5 else 1 for x in y_valid_pred]
f1_score(y_valid, y_valid_pred)



[14:30:13] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


0.1174713787954206

In [ ]:
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")
params_grid = {
    "objective":["reg:logistic"],
    'max_depth':[i for i in range(3,10,2)],
    'min_child_weight':[i for i in range(1,6,2)],
    'gamma':[i/10.0 for i in range(0,5)],
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
    'reg_lambda':[1e-5, 1e-2, 0.1, 1, 100]
}
xgb = XGBClassifier()
gs = GridSearchCV(estimator=xgb, param_grid=params_grid, scoring='f1_weighted')
gs.fit(X_train_bal, y_train_bal)
print(gs.best_score_)